<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [1]:
import azureml.core
from azureml.core import Workspace, Experiment
from azureml.widgets import RunDetails

Failure while loading azureml_run_type_providers. Failed to load entrypoint automl = azureml.train.automl.run:AutoMLRun._from_run_dto with exception (azureml-dataprep 1.6.0 (/opt/conda/lib/python3.7/site-packages), Requirement.parse('azureml-dataprep<1.6.0a,>=1.4.10a'), {'azureml-automl-core'}).


In [2]:
ws = Workspace.from_config()

Performing interactive authentication. Please follow the instructions on the terminal.
To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code CNGUXD5YV to authenticate.
Interactive authentication successfully completed.


In [3]:
experiment = Experiment(workspace = ws, name = "invasive-aml")

In [4]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

cluster_name = "aml-cluster"

# Verify that cluster exists
try:
    training_cluster = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    # If not, create it
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS2_V2', max_nodes=2)
    training_cluster = ComputeTarget.create(ws, cluster_name, compute_config)

training_cluster.wait_for_completion(show_output=True)

Found existing cluster, use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [6]:
from azureml.train.estimator import Estimator
from azureml.train.dnn import TensorFlow

dataset_labels = ws.datasets['fulltrain_labels']
dataset_img = ws.datasets['fulltrain_img']

estimator = TensorFlow(source_directory='./experiments',
                       entry_script='invasive_training.py',
                       compute_target=cluster_name,
                       script_params = {'--img_size': 128, '--epoch': 30, '--batch_size': 32},
                       conda_packages=['scikit-learn', 'matplotlib', 'pillow'],
                       framework_version='2.0',
                       inputs=[dataset_img.as_named_input('images').as_mount(), dataset_labels.as_named_input('labels')]
                      )
run = experiment.submit(estimator)
RunDetails(run).show()

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

In [74]:
help(run)

Help on ScriptRun in module azureml.core.script_run object:

class ScriptRun(azureml.core.run.Run)
 |  ScriptRun(experiment, run_id, directory=None, _run_config=None, **kwargs)
 |  
 |  Provides programmatic access for managing submitted training runs.
 |  
 |  A run submitted with :class:`azureml.core.script_run_config.ScriptRunConfig` represents a single trial in
 |  an experiment. Submitting the run returns a ScriptRun object, which can be used to monitor the asynchronous
 |  execution of the run, log metrics and store output of the run, and analyze results and access artifacts
 |  generated by the run.
 |  
 |  To get started with experiments and ScriptRunConf, see
 |  
 |  * `Create and manage environments for training and
 |    deployment <https://docs.microsoft.com/azure/machine-learning/how-to-use-environments>`_
 |  * `How to manage runs <https://docs.microsoft.com/azure/machine-learning/how-to-manage-runs>`_
 |  
 |  .. remarks::
 |  
 |      The Azure Machine Learning SDK pr

In [76]:
run.get_file_names()

['History_1589474209.png',
 'ROC_1589474216.png',
 'azureml-logs/55_azureml-execution-tvmps_e8907934ee9439588e82cdf74cd51ae363c177a1f5bd13977061cd8b5101d585_d.txt',
 'azureml-logs/65_job_prep-tvmps_e8907934ee9439588e82cdf74cd51ae363c177a1f5bd13977061cd8b5101d585_d.txt',
 'azureml-logs/70_driver_log.txt',
 'azureml-logs/75_job_post-tvmps_e8907934ee9439588e82cdf74cd51ae363c177a1f5bd13977061cd8b5101d585_d.txt',
 'azureml-logs/process_info.json',
 'azureml-logs/process_status.json',
 'logs/azureml/87_azureml.log',
 'logs/azureml/job_prep_azureml.log',
 'logs/azureml/job_release_azureml.log',
 'model']

In [77]:
metrics = run.get_metrics()

In [81]:
metrics

{'image_size': '(64, 64)',
 'batch_size': 32,
 'epochs': 10,
 'History': 'aml://artifactId/ExperimentRun/dcid.invasive-aml_1589473725_75d1a52d/History_1589474209.png',
 'AUC': 0.9255520375015421,
 'ROC': 'aml://artifactId/ExperimentRun/dcid.invasive-aml_1589473725_75d1a52d/ROC_1589474216.png'}

In [85]:
run.register_model(model_path="model",
               model_name="InvasiveCNN",
               tags={'AUC': metrics['AUC'], 'transfer-learning': False,
                     'image_size': metrics['image_size'], 'batch_size': metrics['batch_size'],
                     'epochs': metrics['epochs']},
               description="simple CNN",
               model_framework='TensorFlow',
               model_framework_version='2.1.0',
                  )

Model(workspace=Workspace.create(name='Invasive', subscription_id='68bdd703-8837-469c-80bd-bfb35f3b886f', resource_group='ProjectGroup2'), name=InvasiveCNN, id=InvasiveCNN:1, version=1, tags={'AUC': '0.9255520375015421', 'transfer-learning': 'false', 'image_size': '(64, 64)', 'batch_size': '32', 'epochs': '10'}, properties={})